In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder.appName("CopyMariaDbData")
    .config(
        "spark.jars.packages",
        "mysql:mysql-connector-java:8.0.26,org.apache.spark:spark-sql_2.12:3.1.2",
    ).config(
        "spark.sql.shuffle.partitions",
        10000,
    ).config(
        "spark.executor.memory",
        "4g"
    ).master("spark://spark:7077")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
mysql#mysql-connector-java added as a dependency
org.apache.spark#spark-sql_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b6ba1057-8445-4829-84bf-c54717f454e2;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.26 in central
	found com.google.protobuf#protobuf-java;3.11.4 in central
:: resolution report :: resolve 188ms :: artifacts dl 6ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.11.4 from central in [default]
	mysql#mysql-connector-java;8.0.26 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2

In [3]:
transactions_db: str = "data_warehouse"
transactions_table: str = "transactions"
transactions_url: str = "jdbc:mysql://mariadb:3306/{db}".format(db=transactions_db)
transactions_query: str = "SELECT * FROM {table}".format(table=transactions_table)

transactions_df = (
    spark.read.format("jdbc")
    .option("url", transactions_url)
    .option("driver", "com.mysql.cj.jdbc.Driver")
    .option(
        "query",
        transactions_query,
    )
    .option("user", "root")
    .load()
)

In [4]:
destination_url: str = "jdbc:mysql://mariadb-external:3306/{db}".format(db=transactions_db)

transactions_df.write.format("jdbc") \
    .mode("append") \
    .option("url", destination_url) \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", transactions_table) \
    .option("user", "warehouse") \
    .option("password", "00-wise-ROOT-months-05") \
    .save()

21/10/28 19:02:21 WARN HeartbeatReceiver: Removing executor 0 with no recent heartbeats: 146136 ms exceeds timeout 120000 ms
21/10/28 19:02:21 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.176.6: Executor heartbeat timed out after 146136 ms
21/10/28 19:02:21 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (192.168.176.6 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 146136 ms
21/10/28 19:02:32 WARN TransportChannelHandler: Exception in connection from /192.168.176.6:54890
java.io.IOException: Connection reset by peer
	at java.base/sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:233)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:223)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelIm

KeyboardInterrupt: 

In [ ]:
spark.stop()